# Run baseline

In [1]:
import pickle
import os

unigram_path_dir = '/usr0/home/prashang/DirectedStudy/ACL_preprocessing/Vectors/'

features = {}

with open(os.path.join(unigram_path_dir, 'oneHotFeatures_train.txt'), 'rb') as f:
    features['train'] = pickle.load(f)
    
len(features['train'])

22677

In [2]:
features['train']['1096966_0006_tokenized_paras.txt']

[1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,


## Get tags to predict

In [3]:
import pandas as pd

metadata = pd.read_csv('/usr2/scratch/fanfic/ao3_harrypotter_text/stories.csv')
print(len(metadata))
print(metadata.columns)

179407
Index(['fic_id', 'title', 'author', 'author_key', 'rating', 'category',
       'fandom', 'relationship', 'character', 'additional tags', 'language',
       'published', 'status', 'status date', 'words', 'comments', 'kudos',
       'bookmarks', 'hits', 'chapter_count', 'series', 'seriespart',
       'seriesid', 'summary', 'preface_notes', 'afterword_notes'],
      dtype='object')


In [8]:
metadata.fic_id.dtype

dtype('int64')

In [14]:
len(features['train'])

22677

In [18]:
fic_ids = {}
fic_ids['train'] = sorted(set([int(name.split('_')[0]) for name in list(features['train'].keys())]))
print(len(fic_ids['train']))
fic_ids['train'][:100]

4771
4771


[15427,
 30624,
 35061,
 38866,
 41065,
 41646,
 41647,
 52634,
 52670,
 57428,
 69513,
 102135,
 102138,
 106960,
 112254,
 118750,
 119739,
 154704,
 161250,
 161252,
 161274,
 161276,
 161277,
 178241,
 184767,
 200764,
 246321,
 246340,
 246366,
 246375,
 246388,
 246416,
 252474,
 264944,
 278267,
 278270,
 278297,
 279196,
 285471,
 290499,
 292766,
 293866,
 300899,
 301087,
 309237,
 325442,
 327296,
 328024,
 328836,
 333432,
 339002,
 342146,
 343525,
 349676,
 360880,
 373083,
 374911,
 389578,
 389803,
 390767,
 395351,
 397736,
 398480,
 399222,
 406555,
 409932,
 410533,
 415256,
 419040,
 420987,
 427278,
 429025,
 429814,
 434331,
 436984,
 437697,
 438516,
 442388,
 443908,
 446594,
 446939,
 451266,
 451374,
 456452,
 470422,
 472422,
 479694,
 485385,
 485630,
 487386,
 488789,
 494254,
 501897,
 502159,
 507750,
 508012,
 509845,
 511588,
 512792,
 513412]

In [5]:
# Get tags for training set
tags = {}

train_metadata = metadata.loc[metadata['fic_id'].isin(fic_ids['train']), ['fic_id', 'additional tags']]
train_metadata

tags['train'] = train_metadata.set_index('fic_id').to_dict()['additional tags']
print(len(tags['train'].keys()))

tags['train'][1000154]

4771


'["Cross Over", "Harry potter characters inserted into Supernatural world", "Alternate Universe", "Dark", "Draco is a Demon", "Harry is a Hunter", "Explicit Sexual Content"]'

In [6]:
tags['train'] = {key: [tag.lower() for tag in eval(val)] for key,val in tags['train'].items()}
tags['train'][1000154]

['cross over',
 'harry potter characters inserted into supernatural world',
 'alternate universe',
 'dark',
 'draco is a demon',
 'harry is a hunter',
 'explicit sexual content']

In [10]:
from collections import Counter

tag_ctr = Counter([tag for l in tags['train'].values() for tag in l])
tag_vocab = {}
tag_vocab[100] = [a for a,b in tag_ctr.most_common(100)]
tag_vocab[100]

['fluff',
 'angst',
 'alternate universe - canon divergence',
 'hurt/comfort',
 'slow burn',
 'romance',
 'anal sex',
 'alternate universe',
 'smut',
 'humor',
 'au',
 'fluff and angst',
 'alternate universe - modern setting',
 'established relationship',
 'hogwarts eighth year',
 'hogwarts',
 "marauders' era",
 'time travel',
 'hp: ewe',
 'anal fingering',
 'drarry',
 'happy ending',
 'friendship',
 'friends to lovers',
 'mpreg',
 'crossover',
 'first time',
 'first kiss',
 'pining',
 'oral sex',
 'rimming',
 'angst with a happy ending',
 'wolfstar',
 'alternate universe - hogwarts',
 'blow jobs',
 'other additional tags to be added',
 'kissing',
 'emotional hurt/comfort',
 'magic',
 'implied/referenced child abuse',
 'one shot',
 'quidditch',
 'child abuse',
 'alternate universe - harry potter setting',
 'dumbledore bashing',
 'slash',
 'female harry potter',
 'plot what plot/porn without plot',
 'mutual pining',
 'post-hogwarts',
 'dirty talk',
 'alternate universe - soulmates',
 'p

In [15]:
# How many fics don't have top 100 tags?
notags = [l for l in tags['train'].values() if not any([t in l for t in tag_vocab[100]])]
print(len(notags))
print(len(tags['train']))
print(len(notags)/len(tags['train']))

1100
4771
0.23055963110459024


In [16]:
features['train'].keys()

dict_keys(['1096966_0006_tokenized_paras.txt', '698772_0006_tokenized_paras.txt', '782314_0022_tokenized_paras.txt', '14752551_0002_tokenized_paras.txt', '11740716_0001_tokenized_paras.txt', '9841367_0001_tokenized_paras.txt', '5601274_0026_tokenized_paras.txt', '7312120_0023_tokenized_paras.txt', '12617148_0003_tokenized_paras.txt', '12618536_0003_tokenized_paras.txt', '11923791_0069_tokenized_paras.txt', '6216730_0001_tokenized_paras.txt', '14357595_0001_tokenized_paras.txt', '3982663_0008_tokenized_paras.txt', '2956412_0002_tokenized_paras.txt', '12393252_0013_tokenized_paras.txt', '13376133_0001_tokenized_paras.txt', '9575849_0010_tokenized_paras.txt', '6388831_0004_tokenized_paras.txt', '11211711_0025_tokenized_paras.txt', '13372590_0003_tokenized_paras.txt', '11783307_0015_tokenized_paras.txt', '11557050_0008_tokenized_paras.txt', '1851298_0011_tokenized_paras.txt', '3717433_0015_tokenized_paras.txt', '12042234_0022_tokenized_paras.txt', '8851495_0001_tokenized_paras.txt', '12378

In [ ]:
# Concatenate chapters into fics

In [19]:
# Assemble input features, output
import numpy as np

X_train = np.array([features['train'][f'{fic_id}_tokenized_paras.txt'] for fic_id in fic_ids['train']])
X_train.shape

KeyError: '15427_tokenized_paras.txt'